In [1]:
# Sample user submission
message = '''
    Create a count for each character in the string.
    For each character:
    Convert it to lowercase.
    Count how many times it appears.
    Set total_length to 0 and odd_count to 0.
    For each character count:
    Add the even part of the count to total_length.
    If the count is odd, add 1 to odd_count.
    If odd_count is greater than 0, add 1 to total_length.
    Return total_length.
'''


In [2]:
# Load JSON data
import json

with open('sample_problem.json', 'r') as file:
    problem_data = json.load(file)

# Define system_prompt
prompt = f'''
    You are a professor who specializes in solving code problems.
    The user will submit in words their approach to solving the problem.
    Analyze the user's submission and provide feedback.

    Output your feedback in JSON format with the following structure:
    {{
        "title": "Create a creative title of the analysis",
        "analysis": ["Create a detailed analysis on the user's submission in a list of strings"],
        "suggestions": ["Create suggestions for the user to improve their method in a list of strings"],
        "score": "An integer score: 1 (incorrect), 2 (partially correct), 3 (correct)"
    }}

    Problem Details:
    Category:
    {problem_data['category']}

    Subcategory: 
    {problem_data['subcategory']}

    Difficulty:
    {problem_data['difficulty']}

    Title: 
    {problem_data['title']}

    Description: 
    {problem_data['description']}

    Constraints:
    {chr(10).join([f'{constraint}' for constraint in problem_data['constraints']])}

    Examples:
    {chr(10).join([f'Input: {example['input']}, Output: {example['output']}, Explanation: {example['explanation']}' for example in problem_data['examples']])}

    Here is the user's submission:
    {message}

    Ensure your output only contains the JSON object without any additional text.
    Provide feedback directly addressing the user as ‘you’ instead of referring to them in the third person.
'''


In [3]:
import asyncio
from ollama import AsyncClient

# Must have ollama on in the background
async def ask_model(prompt: str):
    try:
        response = await AsyncClient().generate(model='deepseek-r1:14b', prompt=prompt)
        print(response)
    except Exception as e:
        print(f'🚨 Error: {e}')

asyncio.run(ask_model(prompt))


RuntimeError: asyncio.run() cannot be called from a running event loop